In [ ]:
'''This code will help you confirm if there was growth by adding each carbon source from the PM02 biolog file to the model's carbonless minimal media'''

#Import the necessary libraries.
import cobra
import re
import pandas as pd
from cobra.medium import minimal_medium
from cobra.io import read_sbml_model

#This function extracts the minimal media from the model and extracts the exchange that do not contain a specific element.
def getMinMedias(model, element, model_name):
    '''
    PARAMETERS:
    PARAMETERS:
    model: The model that you want to generate the rich media csv file.
    element: The element you want to exclude from the csv file.
    model_name: A string whos only purpose is to help rename the new generated csv file.'''
    
    min_medium_model = cobra.medium.minimal_medium(model) #Extracting the minimal media from the file.
    matches = re.findall(r"(EX_.+_e)\s", str(min_medium_model)) #Extracting the exchange reactions of the minimal media.
    # We create a variable that will store the exchange reactions of the minimal media. Here I have added other reactions that do not contain carbon that may benefit growth.
    Info = "exchange,uptake\nEX_h2o_e,1000\nEX_h_e,1000\nEX_na1_e,1000\nEX_nh4_e,1000\nEX_ni2_e,1000\nEX_pi_e,1000\n"
    # We iterate through each exchange reaction.
    for medium in matches:
        #print(type(medium))
        # we extract the reaction formula of each exchange reaction.
        reaction = model.reactions.get_by_id(medium)
        #print(medium)
        #print(reaction.metabolites)
        #print(reaction)
        # We extract the metabolite in each of the exchange reactions.
        for metabolite in reaction.metabolites:
            #print(metabolite)
            #print(metabolite.formula)
            #print(element)
            #print("____")
            # Here we extract only the exchange reactions that do not contain the specified element on their metabolite chemical formula.
            if not re.search(str(element), str(metabolite.formula)) and metabolite.formula != None:
                #If the conditions are met, then we add it to the Info variable.
                Info = Info + medium + "," + "1000\n"
                #print(Info)
                #print(metabolite)
                #print(metabolite.formula)
            # This condition also extracts the exchange reactions that do not contain the element but have a similar chemical symbol. In example, while filtering for "C" we do not filter "Ca" or "Cu" elements.
            elif re.search(str(element + "[a-z]"), str(metabolite.formula)):
                #print(metabolite)
                #print(metabolite.formula)
                #If the conditions are met, then we add it to the Info variable.
                Info = Info + medium + "," + "1000\n"
    # We create a new file and write the Info variable in it.
    newfile = open("minimal_media_no_carbon_csc031.csv", "w")
    newfile.write(Info)
    return #print(Info)

#We extract the model and apply the function to it.
model_csc031 = read_sbml_model("/Users/rodr579/Library/CloudStorage/OneDrive-PNNL/venv/GapfilledCSC031.xml")
getMinMedias(model_csc031, "C", "min_medium_csc031")
#min_media_file = pd.read_csv("/Users/rodr579/Library/CloudStorage/OneDrive-PNNL/venv/Testing_New_Media031V2.csv")
#print(min_media_file)

#We open the file that we just created.
min_media_no_carbon = pd.read_csv("/Users/rodr579/Library/CloudStorage/OneDrive-PNNL/venv/minimal_media_no_carbon_csc031.csv")
min_media_dict = {} # We open an empty dictionary.

# We iterate through each exchange reaction found in the file.
for i in range(len(min_media_no_carbon["exchange"])):
    # We add each exchange reaction to the dictionary.
    min_media_dict[min_media_no_carbon["exchange"][i]] = float(min_media_no_carbon["uptake"][i])
# Here are some exchange reactions that we a readding to the dictionary that may benefit growth.
#min_media_dict["EX_glc__D_e"] = 1000.0
min_media_dict["EX_h2o_e"] = 1000.0
min_media_dict["EX_h_e"] = 1000.0
min_media_dict["EX_na1_e"] = 1000.0
min_media_dict["EX_nh4_e"] = 1000.0
min_media_dict["EX_ni2_e"] = 1000.0
min_media_dict["EX_pi_e"] = 1000.0

# We open the file that contains the exchange reactions of each carbon source of the biolog file.
biolog_names = pd.read_csv("/Users/rodr579/Library/CloudStorage/OneDrive-PNNL/venv/PM02OnlyNamesAndBiGG.csv")

# We iterate through each of the biolog reactions.
for reaction in biolog_names["BiGG"]:
    min_media_dict[reaction] = 1000.0 #Here we add each biolog reaction to the media dictionary.
    print("REACTION: " + reaction)
    print("MEDIA: " + str(min_media_dict))
    model_csc031.medium = min_media_dict # We apply this dictionary as the media of the model.
    print("GROWTH: " + str(model_csc031.slim_optimize())) #We calculate the growth of the model in the new media condition.
    print("\n")
    min_media_dict.pop(reaction) #After calculating the growth we remove the current biolog exchange reaction to make room for the next one.

#model_csc031.metabolites.glc__D_e.reactions
#print(model_csc031.reactions.GLCabc.reaction)
#print(model_csc031.reactions.BG_CELLB.reaction)
#print(model_csc031.reactions.GLCpts.reaction)
#med = cobra.medium.minimal_medium(model_csc031)
#print(min_media_dict)
#model_csc031.medium = min_media_dict
#model_csc031.optimize()